In [1]:
import pymc3 as pm
import theano.tensor as tt
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pymc3 as pm
import theano

theano.config.compute_test_value = "off"
pm.set_tt_rng(42)  # Set random seed for TensorFlow or PyTorch

# Load the breast cancer dataset
data = datasets.load_breast_cancer()
X, y = data.data, data.target

# Preprocess the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42)

# Define the Bayesian neural network model using PyMC3
n_hidden = 5

with pm.Model() as neural_network:
    # Define the priors for the weights and biases
    weights_in_1 = pm.Normal("w_in_1", 0, sigma=1,
                             shape=(X.shape[1], n_hidden))
    weights_1_2 = pm.Normal("w_1_2", 0, sigma=1, shape=(n_hidden, n_hidden))
    weights_2_out = pm.Normal("w_2_out", 0, sigma=1, shape=(n_hidden,))
    bias_in_1 = pm.Normal("b_in_1", 0, sigma=1, shape=(n_hidden,))
    bias_1_2 = pm.Normal("b_1_2", 0, sigma=1, shape=(n_hidden,))
    bias_2_out = pm.Normal("b_2_out", 0, sigma=1)

    # Build the neural network architecture
    act_1 = pm.math.tanh(pm.math.dot(X_train, weights_in_1) + bias_in_1)
    act_2 = pm.math.tanh(pm.math.dot(act_1, weights_1_2) + bias_1_2)
    act_out = pm.math.sigmoid(pm.math.dot(act_2, weights_2_out) + bias_2_out)

    # Define the likelihood
    out = pm.Bernoulli("out", act_out, observed=y_train)

    # Perform inference
    trace = pm.sample(1000, cores=1)

# Evaluate the model on the testing set and compute the epistemic uncertainty
with neural_network:
    ppc = pm.sample_posterior_predictive(trace)
    pred = ppc["out"].mean(axis=0) > 0.5
    epistemic_uncertainty = ppc["out"].std(axis=0)
    accuracy = (y_test == pred).mean() * 100

print(f"Accuracy: {accuracy:.2f}%")
print(f"Epistemic uncertainty: {np.mean(epistemic_uncertainty):.4f}")


C:\Users\nstep\AppData\Local\Temp\ipykernel_11096\285632382.py:54: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  accuracy = (y_test == pred).mean() * 100


AttributeError: 'bool' object has no attribute 'mean'

In [2]:
with neural_network:
    ppc = pm.sample_posterior_predictive(trace)
    pred = ppc["out"].mean(axis=0) > 0.5
    epistemic_uncertainty = ppc["out"].std(axis=0)
print(f"Epistemic uncertainty: {np.mean(epistemic_uncertainty):.4f}")

Epistemic uncertainty: 0.1302
